In [1]:
import sys
print(sys.version)
from pycaret.classification import *

3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]


Matplotlib is building the font cache; this may take a moment.


In [2]:
import pandas as pd
from pycaret.classification import *

In [7]:
# 1) Load data
df = pd.read_csv("ckd.csv")
print("Original shape:", df.shape)
display(df.head())

Original shape: (1659, 54)


,PatientID,Age,Gender,Ethnicity,SocioeconomicStatus,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,...,Itching,QualityOfLifeScore,HeavyMetalsExposure,OccupationalExposureChemicals,WaterQuality,MedicalCheckupsFrequency,MedicationAdherence,HealthLiteracy,Diagnosis,DoctorInCharge
0,1,71,0,0,0,2,31.069414,1,5.128112,1.676220,...,7.556302,76.076800,0,0,1,1.018824,4.966808,9.871449,1,Confidential
1,2,34,0,0,1,3,29.692119,1,18.609552,8.377574,...,6.836766,40.128498,0,0,0,3.923538,8.189275,7.161765,1,Confidential
2,3,80,1,1,0,1,37.394822,1,11.882429,9.607401,...,2.144722,92.872842,0,1,1,1.429906,7.624028,7.354632,1,Confidential
3,4,40,0,2,0,1,31.329680,0,16.020165,0.408871,...,7.077188,90.080321,0,0,0,3.226416,3.282688,6.629587,1,Confidential
4,5,43,0,1,1,2,23.726311,0,7.944146,0.780319,...,3.553118,5.258372,0,0,1,0.285466,3.849498,1.437385,1,Confidential


In [6]:
# 2) Drop unnecessary columns
drop_cols = [c for c in ["PatientID", "DoctorInCharge"] if c in df.columns]
df = df.drop(columns=drop_cols)
print("After drop shape:", df.shape)
print("Dropped:", drop_cols)

After drop shape: (1659, 52)
Dropped: ['PatientID', 'DoctorInCharge']


In [8]:
# 3) Check class distribution
print("\nDiagnosis distribution:")
print(df["Diagnosis"].value_counts(dropna=False))


Diagnosis distribution:
Diagnosis
1    1524
0     135
Name: count, dtype: int64


In [10]:
# 4) Setup
clf = setup(
    data=df,
    target="Diagnosis",
    session_id=42,
    train_size=0.8,
    fold=5,
    # preprocessing
    normalize=True,
    fix_imbalance=True, 
    remove_multicollinearity=True,
    multicollinearity_threshold=0.95,
    verbose=True
)

,Description,Value
0,Session id,42
1,Target,Diagnosis
2,Target type,Binary
3,Original data shape,"(1659, 54)"
4,Transformed data shape,"(2770, 54)"
5,Transformed train set shape,"(2438, 54)"
6,Transformed test set shape,"(332, 54)"
7,Numeric features,52
8,Categorical features,1
9,Preprocess,True


In [11]:
# 5) Compare models (choose best by F1)
best_model = compare_models(sort="F1", n_select=1)
print("\nBest model:", best_model)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9427,0.9549,0.9877,0.9518,0.9694,0.5214,0.5494,0.5160
gbc,Gradient Boosting Classifier,0.9390,0.9437,0.9803,0.9545,0.9672,0.5235,0.5358,0.7720
rf,Random Forest Classifier,0.9231,0.9204,0.9967,0.9254,0.9597,0.1477,0.2499,0.2780
qda,Quadratic Discriminant Analysis,0.9194,0.5917,1.0000,0.9193,0.9580,0.0169,0.0420,0.0940
et,Extra Trees Classifier,0.9186,0.8415,1.0000,0.9186,0.9576,0.0000,0.0000,0.1540
ada,Ada Boost Classifier,0.9216,0.8914,0.9606,0.9547,0.9575,0.4497,0.4563,0.2680
dt,Decision Tree Classifier,0.8749,0.7463,0.8999,0.9617,0.9296,0.3682,0.3871,0.0960
nb,Naive Bayes,0.8372,0.7053,0.8860,0.9335,0.9090,0.1380,0.1434,1.6040
lr,Logistic Regression,0.8169,0.8280,0.8294,0.9666,0.8927,0.2917,0.3384,3.4980
svm,SVM - Linear Kernel,0.7890,0.8086,0.8015,0.9626,0.8745,0.2423,0.2904,0.0640



Best model: LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)


In [12]:
# 6) Hyperparameter tuning
tuned_model = tune_model(best_model, optimize="F1")
print("\nTuned model:", tuned_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9398,0.9307,0.9877,0.9488,0.9679,0.5002,0.5266
1,0.9586,0.9605,1.0000,0.9569,0.9780,0.6472,0.6917
2,0.9396,0.9338,0.9877,0.9488,0.9679,0.4712,0.4992
3,0.9547,0.9828,0.9877,0.9640,0.9757,0.6431,0.6536
4,0.9434,0.9448,0.9877,0.9524,0.9697,0.5433,0.5649
Mean,0.9472,0.9505,0.9902,0.9542,0.9718,0.5610,0.5872
Std,0.0079,0.0192,0.0049,0.0057,0.0042,0.0725,0.0738


Fitting 5 folds for each of 10 candidates, totalling 50 fits

Tuned model: LGBMClassifier(bagging_fraction=0.6, bagging_freq=3, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.7,
               importance_type='split', learning_rate=0.2, max_depth=-1,
               min_child_samples=81, min_child_weight=0.001, min_split_gain=0.1,
               n_estimators=90, n_jobs=-1, num_leaves=40, objective=None,
               random_state=42, reg_alpha=0.0005, reg_lambda=0.7, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)


In [13]:
# 7) Evaluate (interactive)
evaluate_model(tuned_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [15]:
# 8) Predictions on holdout
pred_holdout = predict_model(tuned_model)
display(pred_holdout.head())

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.9247,0.9377,0.9639,0.9545,0.9592,0.4692,0.4701


,PatientID,Age,Gender,Ethnicity,SocioeconomicStatus,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,...,HeavyMetalsExposure,OccupationalExposureChemicals,WaterQuality,MedicalCheckupsFrequency,MedicationAdherence,HealthLiteracy,DoctorInCharge,Diagnosis,prediction_label,prediction_score
1232,1233,66,1,1,1,1,32.441158,0,10.534957,0.323453,...,0,0,0,0.650166,8.744802,7.338390,Confidential,1,1,0.9998
1600,1601,58,0,0,1,1,22.454872,0,12.426801,5.691748,...,0,0,0,2.911090,1.838819,2.420768,Confidential,0,0,0.5764
152,153,31,1,1,1,1,36.059284,0,5.044001,6.656737,...,0,0,0,0.787877,6.365757,4.156352,Confidential,1,1,0.9984
491,492,90,1,2,0,2,22.340809,1,7.060485,2.986460,...,0,0,0,2.752223,4.985360,6.824299,Confidential,1,1,0.9949
412,413,32,0,1,2,2,23.505405,0,14.124260,8.877092,...,0,0,0,0.076519,4.752793,2.079430,Confidential,1,1,0.9858


In [16]:
# 9) Metrics table (last pulled results)
results = pull()
display(results)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.9247,0.9377,0.9639,0.9545,0.9592,0.4692,0.4701
